# Оценка баланса реактивной мощности

## Проверка по условию выполнения БРМ в сети и определение приведенных к шинам высшего напряжения нагрузок подстанции 

In [17]:
from IPython.display import display, Math
import numpy as np
%store -r Q_nb_i
%store -r S_nb_i
%store -r Lij

In [18]:
#                  K1    1-3  3-2  3-4   4-5
P_1c = np.array([122.5, 87.5, 35, 37.5, 17.5])
L_lep = np.array([Lij[0, 1], Lij[1, 3], Lij[3, 2], Lij[3, 4], Lij[4, 5]])
mask_p1c = [False, False, True, True, True]

In [19]:
k_odn_q = 0.98
mask_ps = [False, True, False, True, True]
Q_treb_1 = np.around(k_odn_q * np.sum(Q_nb_i), 1)
print(f'Q_treb_1 = {Q_treb_1}')

Q_treb_2 = 0.08 * np.sum(np.where(mask_ps, S_nb_i * 2, S_nb_i))
print(f'Q_treb_2 = {Q_treb_2}')

q_c0_220 = 0.14
q_c0_110 = 0.036
i = 0
Q_c = np.zeros_like(L_lep)
for i in range(len(L_lep)):
    if mask_p1c[i]:
        Q_c[i] = np.around(2 * q_c0_110 * L_lep[i], 2)
    else:
        Q_c[i] = np.around(2 * q_c0_220 * L_lep[i], 2)
print(f'Q_c = {Q_c}')

Q_treb_1 = 93.5
Q_treb_2 = 33.496
Q_c = [11.7  13.78  3.01  1.86  1.86]


In [20]:
P_nat_220 = 130
P_nat_110 = 30
i = 0
delta_Q_l = np.zeros_like(Q_c)
for i in range(len(Q_c)):
    if mask_p1c[i] and P_1c[i] > P_nat_110:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_110)**2 * Q_c[i], 2)
    elif mask_p1c[i] and P_1c[i] <= P_nat_110:
        delta_Q_l[i] = Q_c[i]
    else:
        delta_Q_l[i] = np.around((P_1c[i] / P_nat_220)**2 * Q_c[i], 2)
print(f'delta_Q_l = {delta_Q_l}')

Q_treb_3 = np.around(np.sum(delta_Q_l - Q_c), 1)

Q_treb_sum = np.around(Q_treb_1 + Q_treb_2 + Q_treb_3, 1)
print(f'Q_treb_3 = {Q_treb_3} \nQ_treb_sum = {Q_treb_sum}')

delta_Q_l = [10.39  6.24  4.1   2.91  1.86]
Q_treb_3 = -6.7 
Q_treb_sum = 120.3


## Расстановка КУ

In [21]:
%store -r P_nb
%store -r tg_nb_i
q_rasp = 87
Q_dop_sum = Q_treb_sum - q_rasp
tg_phi_b = np.around((np.sum(Q_nb_i) - Q_dop_sum) / np.sum(P_nb), 3) # высчитываем для всех подстанций

print(tg_phi_b)
print(Q_dop_sum)
print(tg_nb_i)

0.253
33.3
[0.391 0.391 0.377 0.392 0.389]


In [22]:
Q_comp_dop = np.around(P_nb * (tg_nb_i - tg_phi_b), 1)
print("Q_comp_dop до округления= ", Q_comp_dop)

Q_comp_dop = np.array([9.7, 9.7, 3.5, 5.6, 4.8]) # Вручную округляем так, чтобы сходилась сумма Qкуi (Q_comp_dop) и Qку_суммарное (Q_dop_sum) 
print("Q_comp_dop = ", Q_comp_dop)

N_bsk = np.around(Q_comp_dop / 1.2, 1)
N_bsk_treb = np.around(Q_dop_sum / 1.2, 1)
print("N_bsk до округления = ", N_bsk)

N_bsk_sum = 28
print("N_bsk_treb = ", N_bsk_treb)

N_bsk = np.array([8, 8, 2, 6, 4]) # уточняем количество БСК вручную
print("N_bsk_sum = ", N_bsk_sum)
print("N_bsk = ", N_bsk, "\nN_bsk = N_bsk_sum? ---", np.sum(N_bsk) == N_bsk_sum)

Q_comp_dop до округления=  [9.7 9.7 3.7 5.6 4.8]
Q_comp_dop =  [9.7 9.7 3.5 5.6 4.8]
N_bsk до округления =  [8.1 8.1 2.9 4.7 4. ]
N_bsk_treb =  27.8
N_bsk_sum =  28
N_bsk =  [8 8 2 6 4] 
N_bsk = N_bsk_sum? --- True


In [23]:
Q_bsk_dop = N_bsk * 1.2
Q2_nb = np.around(Q_nb_i - Q_bsk_dop, 3) # здесь Q2_nb это Q'_нбi и далее двоечка будет означать апостроф
S2_nb = np.around(np.sqrt(P_nb**2 + Q2_nb**2), 1)
Q_priv = np.around(Q2_nb + 0.08 * S2_nb, 1)
S_priv = np.around(np.sqrt(P_nb**2 + Q_priv**2), 1)

Q_priv[3-1] = np.around(Q2_nb[3-1] + Q_priv[3] + Q_priv[4] + Q_priv[1] + 0.08 * 
                        np.sqrt((P_nb[2] + P_nb[3] + P_nb[4] + P_nb[1])**2 + (Q2_nb[3-1] + Q_priv[3] + Q_priv[4] + Q_priv[1])**2), 1)
S_priv[3-1] = np.around(np.sqrt((P_nb[2] + P_nb[3] + P_nb[4] + P_nb[1])**2 + Q_priv[2]), 1)

print("Q_bsk_dop = ", Q_bsk_dop, "\nQ''_nb = ", Q2_nb, "\nS''_nb = ", S2_nb, "\nQ_priv = ", Q_priv, "\nS_priv = ", S_priv)

Q_bsk_dop =  [9.6 9.6 2.4 7.2 4.8] 
Q''_nb =  [17.8 17.8  8.9  8.5  8.8] 
S''_nb =  [72.2 72.2 31.3 40.9 36.1] 
Q_priv =  [23.6 23.6 70.7 11.8 11.7] 
S_priv =  [ 73.9  73.9 175.2  41.7  36.9]


In [24]:
display(Math(f'''Q_{{прив3}} = Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}} + Q_{{прив2}} + 0,08 \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{нб5}} + P_{{нб2}})^2 + (Q_{{нб3}}^{{''}} + Q_{{прив4}} + Q_{{прив5}} + Q_{{прив2}})^2}} = 
             {Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]} + {Q_priv[1]} + 0,08 \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]} + {P_nb[1]})^2 + ({Q2_nb[3-1]} + {Q_priv[3]} + {Q_priv[4]} + {Q_priv[1]})^2}} = {Q_priv[3-1]}'''))

<IPython.core.display.Math object>

In [25]:
display(Math(f'''S_{{прив3}} = \sqrt{{(P_{{нб3}} + P_{{нб4}} + P_{{нб5}} + P_{{нб2}})^2 + (Q_{{прив3}})^2}} = 
                \sqrt{{({P_nb[2]} + {P_nb[3]} + {P_nb[4]} + {P_nb[1]})^2 + ({Q_priv[2]})^2}} = {S_priv[3-1]}'''))

<IPython.core.display.Math object>

In [26]:
print(np.sum(Q2_nb))

61.8


# Выбор сечений проводов линий электропередачи и их проверка по условиям технических ограничений

In [27]:
j_ek = 0.9
alpha_i = 1.05
#                K1   1-3    3-2   3-4  4-5
Q = np.array([141.8, 117.8, 23.6, 23.5, 11.7])
P = np.copy(P_1c) * 2

i = 0
I_nb_5 = np.zeros_like(L_lep)
for i in range(len(L_lep)):
    if mask_p1c[i]:
        I_nb_5[i] = np.around(np.sqrt(P[i]**2 + Q[i]**2) / (np.sqrt(3) * 110 * 2) * 1000, 1)
    else:
        I_nb_5[i] = np.around(np.sqrt(P[i]**2 + Q[i]**2) / (np.sqrt(3) * 220 * 2) * 1000, 1)

I_r = alpha_i * I_nb_5
F_ek = np.around(I_r / j_ek, 1)

print(f'P = {P} \nQ = {Q} \nS = {np.around(np.sqrt(P**2 + Q**2), 1)}')
print("I_нб_(5) = ", I_nb_5, "\nI_р = ", I_r, "\nF_эк = ", F_ek)

P = [245. 175.  70.  75.  35.] 
Q = [141.8 117.8  23.6  23.5  11.7] 
S = [283.1 211.   73.9  78.6  36.9]
I_нб_(5) =  [371.4 276.8 193.9 206.3  96.8] 
I_р =  [389.97  290.64  203.595 216.615 101.64 ] 
F_эк =  [433.3 322.9 226.2 240.7 112.9]


## Оценка технической осуществимости схемы

In [28]:
R_0 = np.array([0.073, 0.096, 0.118, 0.118, 0.244])
X_0 = np.array([0.42, 0.429, 0.405, 0.405, 0.427])
B_0 = np.array([2.701, 2.645, 2.808, 2.808, 2.658])

In [29]:
R = np.around(R_0 * L_lep / 2, 2)
X = np.around(X_0 * L_lep / 2, 2)
B = np.around(B_0 * L_lep * 2 / 2, 2)

In [30]:
print(f'R = {R} \nX = {X} \nB/2 = {B}') 

R = [1.53 2.36 2.47 1.53 3.16] 
X = [ 8.78 10.55  8.46  5.24  5.53] 
B/2 = [112.9  130.13 117.37  72.73  68.84]


# Выбор трансформаторов и автотрансформаторов на подстанциях

In [31]:
k_av = 1.4
n_t = 2
S_t = np.around(S2_nb / (k_av * (n_t -1)), 1)
S_t = np.delete(S_t, 2)
print(f"S''_nb = {S2_nb} \nS_t = {S_t}")

S_at_1 = np.around((np.sqrt(P_nb[2]**2 + Q2_nb[2]**2) + np.sqrt(P_nb[3]**2 + Q_priv[3]**2) + np.sqrt(P_nb[4]**2 + Q_priv[4]**2)) / (1.2 * (n_t - 1)), 1)
S_at_2 = np.around(S2_nb[2] / (1.2 * 0.5 * (n_t - 1)), 1)
print(f"S_ат_1 = {S_at_1} \nS_ат_2 = {S_at_2}")

S_lrt = np.around(S2_nb[2] / 1.4, 1)
print('S_лрт = ', S_lrt)

S''_nb = [72.2 72.2 31.3 40.9 36.1] 
S_t = [51.6 51.6 29.2 25.8]
S_ат_1 = 91.6 
S_ат_2 = 52.2
S_лрт =  22.4
